Importing the necessaries libraries to work with

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn import cluster

Reading the data

In [7]:
dataSet = pd.read_csv("Guilherme Antonio Aguiar Pessanha Santos - vehicle_1Mb.csv", header=None)
dataSet.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,NaN,id,url,region,region_url,price,year,manufacturer,model,condition,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
1,58589.0,7303354444,https://slo.craigslist.org/cto/d/nipomo-2008-f...,san luis obispo,https://slo.craigslist.org,7000,2008.0,ford,f150,good,...,NaN,pickup,NaN,https://images.craigslist.org/00w0w_abA4EveUZw...,This is a Green 2008 FORD F-150 XL 4 DOOR ext...,NaN,ca,350.298,-1.204.894,2021-04-07T14:28:18-0700
2,98736.0,7314126068,https://jacksonville.craigslist.org/cto/d/sain...,jacksonville,https://jacksonville.craigslist.org,17500,2016.0,bmw,328i,like new,...,mid-size,sedan,blue,https://images.craigslist.org/00e0e_fks755tyfI...,2016 Bmw 328i. 63000 miles. Second owner. Serv...,NaN,fl,300.815,-815.477,2021-04-28T20:14:28-0400
3,401980.0,7303685904,https://bellingham.craigslist.org/ctd/d/puyall...,bellingham,https://bellingham.craigslist.org,44999,2019.0,ram,1500,NaN,...,NaN,truck,white,https://images.craigslist.org/00U0U_iIvcoJZBwq...,Joydrive: 253-201-0804 https://joydrive.com/...,NaN,wa,471.991,-1.223.151,2021-04-08T09:17:55-0700
4,257546.0,7311166305,https://southjersey.craigslist.org/cto/d/egg-h...,south jersey,https://southjersey.craigslist.org,3990,2002.0,honda,cr-v,good,...,NaN,NaN,NaN,https://images.craigslist.org/01717_eNGBPppC61...,"2002 Honda CR-V 6 cylinder, FWD Clean title. ...",NaN,nj,393.741,-746.118,2021-04-22T21:15:18-0400


Seeing its info

In [8]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       304 non-null    float64
 1   1       305 non-null    object 
 2   2       305 non-null    object 
 3   3       305 non-null    object 
 4   4       305 non-null    object 
 5   5       305 non-null    object 
 6   6       305 non-null    object 
 7   7       297 non-null    object 
 8   8       305 non-null    object 
 9   9       193 non-null    object 
 10  10      183 non-null    object 
 11  11      303 non-null    object 
 12  12      298 non-null    object 
 13  13      296 non-null    object 
 14  14      302 non-null    object 
 15  15      201 non-null    object 
 16  16      211 non-null    object 
 17  17      90 non-null     object 
 18  18      244 non-null    object 
 19  19      210 non-null    object 
 20  20      305 non-null    object 
 21  21      305 non-null    object 
 22  22

The features of numbers 0, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 24, 25 have data missing init. I will remove the feature of number 2 because it only has 1 non-Null sata init

Discrbing the features

In [9]:
dataSet.describe()

,0
count,304.000000
mean,207976.680921
std,119573.752387
min,2933.000000
25%,98635.500000
50%,208630.000000
75%,311334.750000
max,426484.000000


The feature of number 22 which doesn't have any data init 

In [23]:
dataSet[22]

0      county
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
300       NaN
301       NaN
302       NaN
303       NaN
304       NaN
Name: 22, Length: 305, dtype: object

In [49]:
dataSet[17]

0          size
1           NaN
2      mid-size
3           NaN
4           NaN
         ...   
300         NaN
301         NaN
302         NaN
303         NaN
304         NaN
Name: 17, Length: 305, dtype: object

The feature is county and only has one non-null data, so now i will remove the the feature from the dataSet 

In [73]:
dataSet[17].keys()


RangeIndex(start=0, stop=305, step=1)

In [60]:
dataSet_cleaned = dataSet.copy()
# dataSet1.drop("county",axis = 22)
# deleting the county feature
del dataSet_cleaned[22]
# deleting the size feature
del dataSet_cleaned[17]
del dataSet_cleaned[2]
del dataSet_cleaned[4]
del dataSet_cleaned[20]
dataSet_cleaned

,0,1,3,5,6,7,8,9,10,11,...,14,15,16,18,19,21,23,24,25,26
0,NaN,id,region,price,year,manufacturer,model,condition,cylinders,fuel,...,transmission,VIN,drive,type,paint_color,description,state,lat,long,posting_date
1,58589.0,7303354444,san luis obispo,7000,2008.0,ford,f150,good,NaN,gas,...,automatic,NaN,NaN,pickup,NaN,This is a Green 2008 FORD F-150 XL 4 DOOR ext...,ca,350.298,-1.204.894,2021-04-07T14:28:18-0700
2,98736.0,7314126068,jacksonville,17500,2016.0,bmw,328i,like new,4 cylinders,gas,...,automatic,NaN,rwd,sedan,blue,2016 Bmw 328i. 63000 miles. Second owner. Serv...,fl,300.815,-815.477,2021-04-28T20:14:28-0400
3,401980.0,7303685904,bellingham,44999,2019.0,ram,1500,NaN,8 cylinders,gas,...,automatic,1C6SRFETXKN545463,4wd,truck,white,Joydrive: 253-201-0804 https://joydrive.com/...,wa,471.991,-1.223.151,2021-04-08T09:17:55-0700
4,257546.0,7311166305,south jersey,3990,2002.0,honda,cr-v,good,NaN,gas,...,automatic,NaN,NaN,NaN,NaN,"2002 Honda CR-V 6 cylinder, FWD Clean title. ...",nj,393.741,-746.118,2021-04-22T21:15:18-0400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,80436.0,7304367513,fort collins / north CO,24900,2016.0,ford,f250 super duty,excellent,8 cylinders,gas,...,automatic,NaN,4wd,pickup,white,"2016 Ford F250 XLT SuperCab 4x4, 6.2L V8 gas e...",co,40.601.103,-10.507.663,2021-04-09T14:41:08-0600
301,58144.0,7308327114,san luis obispo,14777,2015.0,volvo,xc60 local lots of records,excellent,NaN,gas,...,automatic,YV426MDC0F2593243,fwd,SUV,NaN,2015 *** Volvo XC60 LOCAL LOTS OF RECORDS FWD ...,ca,35.283.445,-120.658.029,2021-04-17T10:15:51-0700
302,176811.0,7307677031,new orleans,7200,2012.0,toyota,corolla,excellent,4 cylinders,gas,...,automatic,NaN,fwd,NaN,NaN,Very Dependable 4cyl Toyota!! Runs Great! Exce...,la,30.493.374,-90.427.694,2021-04-16T09:16:20-0500
303,422865.0,7314851341,milwaukee,3600,1995.0,ford,f-150,excellent,8 cylinders,gas,...,automatic,NaN,rwd,truck,purple,1995 Ford f150 for sale. 5.0 automatic RWD. Li...,wi,429.558,-87.993,2021-04-30T11:16:24-0500


The first feature has only one row no-null and this feature is the value of the vehicle

In [13]:
def split_train_test(data, test_ratio):
  "splits the data"
  shuffled_indices = np.random.permutation(len(data))
  test_set_size = int(len(data) * test_ratio)
  test_indices = shuffled_indices[:test_set_size]
  train_indices = shuffled_indices[test_set_size:]
  return data.iloc[train_indices], data.iloc[test_indices]

In [16]:
 train_set, test_set = split_train_test(dataSet, 0.2)
 print(len(train_set))
 print(len(test_set))
 test_set.head()

244
61


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
23,221108.0,7307499641,https://loz.craigslist.org/cto/d/macks-creek-r...,lake of the ozarks,https://loz.craigslist.org,29000,2017.0,jeep,wrangler unlimited,good,...,NaN,SUV,blue,https://images.craigslist.org/00O0O_1KpjdiA0fb...,"2017 Jeep Wrangler · Suv · Driven 95,000 miles...",NaN,mo,379.616,-929.603,2021-04-15T19:20:13-0500
175,245372.0,7316220734,https://lincoln.craigslist.org/ctd/d/columbus-...,lincoln,https://lincoln.craigslist.org,28888,2018.0,lincoln,mkc,NaN,...,NaN,SUV,NaN,https://images.craigslist.org/00Z0Z_ghDK5BIHT7...,"2018 Lincoln MKC Select AWD -- $28,888 ☎...",NaN,ne,41.438.366,-97.316.899,2021-05-03T10:35:07-0500
167,358067.0,7306800052,https://memphis.craigslist.org/ctd/d/memphis-2...,memphis,https://memphis.craigslist.org,10995,2012.0,toyota,camry,NaN,...,NaN,sedan,white,https://images.craigslist.org/00I0I_7HMlYjXP9E...,"2012 *Toyota* *Camry* Sedan - $10,995Call Us ...",NaN,tn,35.191.437,-89.894.973,2021-04-14T13:30:21-0500
290,93226.0,7305668893,https://daytona.craigslist.org/ctd/d/orlando-2...,daytona beach,https://daytona.craigslist.org,1,2017.0,ford,f-150,excellent,...,full-size,truck,white,https://images.craigslist.org/00v0v_4adamixylj...,CREDIT AMNESTY! SE HABLA ESPANOL !!OPEN SUNDA...,NaN,fl,28.466.088,-81.396.429,2021-04-12T12:15:01-0400
291,85572.0,7304630826,https://hartford.craigslist.org/ctd/d/hartford...,hartford,https://hartford.craigslist.org,24990,2017.0,lexus,es 350 sedan 4d,good,...,NaN,sedan,NaN,https://images.craigslist.org/00f0f_MGjKA4gkdk...,Carvana is the safer way to buy a car During t...,NaN,ct,41.76,-72.68,2021-04-10T09:51:21-0400
